---
title: plotly + ipywidgets volume rendering
author: Georgios Varnavides
date: 2023/07/30
---

This is an example widget illustrating how to use plotly and ipywidgets together for 3D volume rendering.

In [34]:
#| label: app:plotly_interactive_ipywidgets
# Widget combining plotly + ipywidgets for rendering.

%matplotlib widget

import plotly.graph_objects as go
from IPython.display import display
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, FloatRangeSlider, Layout, HBox, VBox
import numpy as np
import h5py

# Data
with h5py.File("data/CNT_overlap_tomo_missing.h5","r") as f:
    values = f['reconstruction'][::2,::2,::2]
    values /= values.max()
    values = np.log(values+1)

nx, ny, nz = values.shape
X, Y, Z = np.mgrid[0:nx,0:ny,0:nz]
dpi = 70

# Histogram visualization
hist_range_plot = (-4,4)
hist_range_init = (-1,2)
hist_num_bins = 200

plt.ioff()
def compute_histogram_values(array):
    
    int_mean = np.mean(array)
    int_std = np.sqrt(np.mean((array - int_mean)**2))
    
    int_min = int_mean + hist_range_plot[0] * int_std
    int_max = int_mean + hist_range_plot[1] * int_std
    
    init_min = np.maximum(int_mean + hist_range_init[0] * int_std,0)
    init_max = np.minimum(int_mean + hist_range_init[1] * int_std,1)
    
    int_ranges = (int_mean, int_std, int_min, int_max,init_min,init_max)
    
    hist_bins = np.linspace(
        int_ranges[2],
        int_ranges[3],
        hist_num_bins+1,
        endpoint=True)
    hist_data, _ = np.histogram(
        array.ravel(),
        bins=hist_bins,
    )
    hist_data = hist_data.astype('float')
    hist_data /= np.max(hist_data)
    
    hist_bins = hist_bins[:-1] + (hist_bins[1] - hist_bins[0])/2
    
    return hist_bins, hist_data, int_ranges

hist_bins, hist_data, int_ranges = compute_histogram_values(values)
fig_hist, ax_hist = plt.subplots(figsize=(175/dpi, 150/dpi), dpi=dpi)

h_hist = ax_hist.fill_between(
    hist_bins,
    hist_data,
    color = (0, 0.7, 1.0, 1.0),
)

h_vlines =ax_hist.vlines(
    int_ranges[4:6],
    ymin = 0,
    ymax = 1.1,
    color = 'k',
)

ax_hist.set_xlim((int_ranges[2], int_ranges[3]));
ax_hist.set_ylim((0, 1.1));
ax_hist.set(yticks=[])
ax_hist.set(yticklabels=[])

fig_hist.canvas.toolbar_visible = False
fig_hist.canvas.header_visible = False
fig_hist.canvas.footer_visible = False
fig_hist.canvas.resizable = False
fig_hist.tight_layout()

config = {
    'scrollZoom':False,
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': [
        'orbitRotation',
        'resetCameraLastSave3d',
        'pan'
    ],
}

layout = go.Layout(
        scene={
        'xaxis':{
            'visible':False
        },
        'yaxis':{
            'visible':False
        },
        'zaxis':{
            'visible':False
        },
        'aspectmode':'cube',
    },
    width=400,
    height=400,
    autosize=False,
)

fig = go.FigureWidget(
        data=go.Volume(
            
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=values.flatten(),
                
            isomin=-0.1,
            isomax=0.8,
            opacity=0.1,
            surface_count=8,
            showscale=True,
            xhoverformat=".3f",
            yhoverformat=".3f",
            zhoverformat=".3f",
            valuehoverformat=".3f",
            cmin=int_ranges[4],
            cmax=int_ranges[5],

            colorbar={
                'thicknessmode': "fraction",
                'thickness': 0.025,
            },
            colorscale='Greys',
            
        ),
    layout=layout,
)

fig._config = fig._config | config

# Widgets 
sequential_cmaps = [
    'viridis', 'plasma', 'inferno', 'magma', 'cividis','turbo',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
]

def update_colormap(change):
    cmap = change['new']
    fig.data[0].colorscale = cmap
    return None

cmap_widget =Dropdown(options=sequential_cmaps,value='Greys',description="Colormap",indent=False,layout=Layout(width='175px'))
cmap_widget.observe(update_colormap,names='value')


def update_vlines(change):
    min, max = change['new']
    p = h_vlines.get_segments()
    p = np.array([
        [
            [min, 0],
            [min, 1.1],
        ],
        [
            [max, 0],
            [max, 1.1],
        ]
    ])
    h_vlines.set_segments(p)
    fig_hist.canvas.draw_idle()
    
    fig.data[0].cmin = min
    fig.data[0].cmax = max
    
    return None

histogram_range_slider = FloatRangeSlider(
    value=int_ranges[4:6],
    min=int_ranges[2],
    max=int_ranges[3],
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    indent=True,
    layout=Layout(width='175px')
)
histogram_range_slider.observe(update_vlines,names='value')

visualization_layout = Layout(
    display='flex',
    flex_flow='row',
    align_items='center',
    width='600px'
)

histogram_box_layout = Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    width='200px'
)
histogram_vbox = VBox([fig_hist.canvas,histogram_range_slider],layout=histogram_box_layout)
controls_vbox = VBox([histogram_vbox,cmap_widget],layout=histogram_box_layout)

display(
    HBox(
        [
            fig,
            controls_vbox
        ],
        layout=visualization_layout
        )
)

    'data': [{'cmax': 0.38065817207098007,
              'cmin': 0.0,
          …